## Regression Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shapely
import shapely.speedups
from sklearn import linear_model
import geopandas as gpd
import math
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

/home/shrey/.local/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
# Data -- Site 1
dfout_s1 = pd.read_csv('Data/PVData/Site1/PVOut_46834.csv', header=None)
dfout_s1 = dfout_s1.drop(axis=1, columns=[3,4,6,7,8,9,10,11,12,13])
dfout_s1 = dfout_s1.drop(axis=0, index=[item for item in range(32,49+1)])
dfout_s1 = dfout_s1.drop(axis=0, index=[0])
dfout_s1 = dfout_s1.loc[::-1].reset_index(drop=True)

dfsys_s1 = pd.read_csv('Data/PVData/Site1/PVSystem_46834.csv', header=None)

weather_s1 = pd.read_csv('Data/PVData/Site1/NSRDBout_s1.csv', header=None)
weather_s1 = weather_s1.drop(axis=0, index=[0,1,2])
weather_s1 = weather_s1.drop(axis=1, columns=[item for item in range(14,47)])

In [3]:
# Data -- Site 2
dfout_s2 = pd.read_csv('Data/PVData/Site2/PVOut_3445.csv', header=None)
dfout_s2 = dfout_s2.drop(axis=1, columns=[3,4,6,7,8,9,10,11,12,13])
dfout_s2 = dfout_s2.loc[::-1].reset_index(drop=True)

dfsys_s2 = pd.read_csv('Data/PVData/Site2/PVSystem_3445.csv', header=None)

weather_s2 = pd.read_csv('Data/PVData/Site2/NSRDBout_s2.csv', header=None)
weather_s2 = weather_s2.drop(axis=0, index=[0,1,2])
weather_s2 = weather_s2.drop(axis=1, columns=[item for item in range(14,47)])

In [4]:
def outputPanel(systemCapacity,energyGen):
    # We are calculating the output per panel assuming that each panel has a capactiy of 300 Watts
    wattage = 300
    newPanel = systemCapacity/wattage
    outPerPanel = energyGen/newPanel
    
    return int(round(int(outPerPanel)))

In [5]:
# Returns daily averages for meteorlogical data over the course of a specified month
## also added the energy output to this dataset
# One must specify the MONTH and the NUMBER OF DAYS IN THE MONTH
def daily(weatherData,pvOutData,pvSysData,year,month,numOfDaysMonth):
    
    monthstr = str(month)
    
    # Start of with monthly dataset
    mask = weatherData[2].values == monthstr
    pos = np.flatnonzero(mask)
    monthData = weatherData.iloc[pos]

    dailyData = pd.DataFrame(columns = ['Year','Month','Day','lat','lon','GHI','DHI','DNI',
                                        'Energy Gen (watt hr)',
                                        'Wind Speed','Temperature','Solar Zenith Angle',
                                        'Pressure','Relative Humidity'])    
        
    # Now the Daily Datasets
    for i in range(1, numOfDaysMonth+1):
        # Weather Data
        dayMask = monthData[3].values == str(i)
        pos = np.flatnonzero(dayMask)
        dayData = monthData.iloc[pos]
            
        #PV Data
        gen = outputPanel(pvSysData.iloc[0,1], pvOutData.iloc[(i-1),1])
        
        #PV and Weather Data combined
        df2 = pd.DataFrame([{'Year':year, 'Month':month, 'Day':i, 
                             'lat':pvSysData.iloc[0,13],'lon':pvSysData.iloc[0,14],
                             'Energy Gen (watt hr)':gen, 
                             'GHI':average(dayData,6), 'DHI':average(dayData,7), 
                             'DNI':average(dayData,8), 'Wind Speed':average(dayData,9), 
                             'Temperature':average(dayData,10), 'Solar Zenith Angle':average(dayData,11),
                             'Pressure':average(dayData,12), 'Relative Humidity':average(dayData,13)}])
        
        dailyData = dailyData.append(df2, ignore_index=True)
        
    return dailyData

In [8]:
def average(data, col):
    if col == int:
        col = int(col)
    elif col == str:
        col = str(col)
    co = data[col].tolist()
    intList = [float(item) for item in co]
    avg = sum(intList)/len(intList)
    return avg

In [23]:
dataset_s1  = daily(weather_s1,dfout_s1,dfsys_s1,2020,5,31)
dataset_s2 = daily(weather_s2,dfout_s2,dfsys_s2,2020,6,30)
dataset = dataset_s1.append(dataset_s2)
dataset

,Year,Month,Day,lat,lon,GHI,DHI,DNI,Energy Gen (watt hr),Wind Speed,Temperature,Solar Zenith Angle,Pressure,Relative Humidity
0,2020,5,1,32.855817,-116.952878,325.083333,55.666667,398.250000,1611,2.395833,20.162500,79.019583,998.333333,63.931250
1,2020,5,2,32.855817,-116.952878,342.333333,51.625000,423.291667,1840,2.583333,19.758333,78.815417,1000.375000,64.340417
2,2020,5,3,32.855817,-116.952878,342.708333,53.458333,419.791667,1820,2.591667,19.287500,78.615833,1000.958333,66.843333
3,2020,5,4,32.855817,-116.952878,348.625000,52.666667,428.958333,1849,1.520833,22.100000,78.420000,999.875000,53.775833
4,2020,5,5,32.855817,-116.952878,348.958333,52.958333,430.541667,1822,1.487500,25.558333,78.227500,999.583333,42.432917
5,2020,5,6,32.855817,-116.952878,352.458333,54.291667,441.250000,1815,1.433333,26.129167,78.038750,998.250000,40.418333
6,2020,5,7,32.855817,-116.952878,352.583333,52.500000,439.666667,1828,1.341667,25.141667,77.851667,996.875000,44.829167
7,2020,5,8,32.855817,-116.952878,343.375000,55.541667,419.708333,1830,1.333333,23.937500,77.668750,997.458333,47.859583
8,2020,5,9,32.855817,-116.952878,304.375000,71.625000,312.291667,1580,1.054167,22.470833,77.489167,998.208333,60.938750
9,2020,5,10,32.855817,-116.952878,238.875000,74.291667,259.791667,1145,2.566667,20.370833,77.313750,998.791667,66.915417


## Multiple Regression Model

In [10]:
# load in variables
x = dataset[['GHI','Relative Humidity','Pressure','Temperature','Solar Zenith Angle','Wind Speed','DNI','DHI']]
y = dataset['Energy Gen (watt hr)']

In [24]:
## Load in regression model
regr = linear_model.LinearRegression()
regr.fit(x,y)

# r^2 value
r2 = regr.score(x,y)
print('r squared: ',r2, '\nVariables: ', x.columns.to_list())


r squared:  0.930235077024612 
Variables:  ['GHI', 'Relative Humidity', 'Pressure', 'Temperature', 'Solar Zenith Angle', 'Wind Speed', 'DNI', 'DHI']


## Prediction Data

In [21]:
# setup -- getting rid of states that are not in the contiguous US

pd.set_option('display.max_columns', None)
shapely.speedups.enable()
# Get shape file of Continguous US
df = gpd.read_file('CartographicBoundries/US_State/cb_2018_us_state_500k.shp')
df = df.drop([37,38,44,45,13,27,42])
contUSdf = df.dissolve()

## upper and lower bound of coords in the contiguous US
bound = pd.DataFrame(contUSdf.bounds)

In [22]:
# Return a list of the boundries from the Polygon
def find_boundary(index):
    minx = bound.iat[index, 0]
    miny = bound.iat[index, 1]
    maxx = bound.iat[index, 2]
    maxy = bound.iat[index, 3]
    return minx, miny, maxx, maxy

In [15]:
# This function should be named 'factor' ... but whatever
def multiples(n):
    l = [*range(1, n+1)]
    multiples = []
    for i in range(1, l[len(l)-1]+1):
        if n % i == 0:
            multiples.append(i)
    return multiples

In [17]:
# Find the number of subdivisions to make,
# Uses the middle 2 elements of the mutiples list ...
# ... (created by above function)
def num_of_divisions(area):
    m = multiples(area)
    w_n = m[len(m) // 2]
    h_n = m[len(m) // 2]
    if (len(m) % 2) == 1:
        return w_n, h_n
    else:
        h_n = m[(len(m) // 2) - 1]
    return w_n, h_n

In [16]:
# length of divisions .. calculated using the diffrence in coordinate values
# ... therefore, not standard for both latitude and longitude
def len_of_div(bounds, width_n, height_n):
    a = bounds[0]
    # width subdivision
    width = bounds[2] - bounds[0]
    length_of_each_div_w = width/width_n
    # length subdivision
    height = bounds[3] - bounds[1]
    length_of_each_div_h = height/height_n
    return length_of_each_div_w, length_of_each_div_h

In [18]:
# returns list of coords to use as datapoints, each respective position in the lat and lon corresponds to the same datapoint
## Future - - perhaps make a sigular list with tuples representing each datapoint rather than haveing two datasets (one for lat and other for lon)
def coord_of_div(len_div, bounds, num_of_divs):
    ## formula for first point = a + (1/2)length
    initial_coord_w = bounds[0] + (0.5 * len_div[0])
    initial_coord_h = bounds[1] + (0.5 * len_div[1])
    w = []
    h = []
    for i in range (num_of_divs[0]):
        w.append(initial_coord_w + (i * len_div[0]))
    for i in range (num_of_divs[1]):
        h.append(initial_coord_h + (i * len_div[1]))
    return w, h

In [19]:
# Filters out datapoint that are not in the contiguous USA
def point_dataframe(longitude, latitude):
    point = pd.DataFrame(columns = ['lon', 'lat'])
    for x in longitude:
        for y in latitude:
            point2 = pd.DataFrame([{'lon':x, 'lat':y}])
            point = point.append(point2, ignore_index = True)
    gdf = gpd.GeoDataFrame(point,
            geometry = gpd.points_from_xy(point.lon, point.lat))
    ## Check if point is in main polygon
    # pip = 'point in polygon'
    pip_mask = gdf.within(contUSdf.loc[0, 'geometry'])
    pip_data = gdf.loc[pip_mask]
    return pip_data

In [20]:
# using the NSRDB API to get weather data for wach of the points, yearly averages
# future notes -- plan on getting monthly averages
def get_data(list_lat, list_lon):
    
    df = pd.DataFrame(columns = ['lat','lon','GHI','DHI','DNI',
                                 'Wind Speed','Temperature',
                                 'Solar Zenith Angle','Pressure',
                                 'Relative Humidity'])
    
    if len(list_lat) != len(list_lon):
        print("Error -- shape of list_lat does not match that of list_lons")
    else:
        for i in range(len(list_lat)):
                # Declare all variables as strings. Spaces must be replaced with '+', i.e., change 'John Smith' to 'John+Smith'.
                # Define the lat, long of the location and the year
                latitude = list_lat[i]
                longitude = list_lon[i]
                lat,lon,year = latitude, longitude, 2010
                # You must request an NSRDB api key from the link above
                api_key = 'Ys1FBygszkOmc2ifUvWD8LdkRFWGaIbNByDa5Ddc'
                # Set the attributes to extract (e.g., dhi, ghi, etc.), separated by commas.
                attributes = 'ghi,dhi,dni,wind_speed,air_temperature,solar_zenith_angle,surface_pressure,relative_humidity'            # Choose year of data
                year = '2020'
                # Set leap year to true or false. True will return leap day data if present, false will not.
                leap_year = 'false'
                # Set time interval in minutes, i.e., '30' is half hour intervals. Valid intervals are 30 & 60.
                interval = '60'
                # Specify Coordinated Universal Time (UTC), 'true' will use UTC, 'false' will use the local time zone of the data.
                # NOTE: In order to use the NSRDB data in SAM, you must specify UTC as 'false'. SAM requires the data to be in the
                # local time zone.
                utc = 'true'
                # Your full name, use '+' instead of spaces.
                your_name = 'Shrey+Poshiya'
                # Your reason for using the NSRDB.
                reason_for_use = 'personal+project'
                # Your affiliation
                your_affiliation = 'Santa+Fe+Preparatory+School'
                # Your email address
                your_email = 'shreyposh@gmail.com'
                # Please join our mailing list so we can keep you up-to-date on new developments.
                mailing_list = 'false'
                # Declare url string
                url = 'https://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api_key, attr=attributes)
                # Return just the first 2 lines to get metadata:
                info = pd.read_csv(url, skiprows=2)
                # See metadata for specified properties, e.g., timezone and elevation
                info = info.drop(axis=0, index=[0])
                print(info)

                ## Make a new dataframe, with the same columns as the first dataframe ... then concat
                df2 = pd.DataFrame([{'lat':latitude, 'lon':longitude, 'GHI':average(info, 'GHI'), 
                                     'DHI':average(info, 'DHI'), 
                                     'DNI':average(info, 'DNI'),
                                     'Wind Speed':average(info, 'Wind Speed'),
                                     'Temperature':average(info, 'Temperature'),
                                     'Solar Zenith Angle':average(info, 'Solar Zenith Angle'),
                                     'Pressure':average(info, 'Pressure'),
                                     'Relative Humidity':average(info, 'Relative Humidity')}])
                df = df.append(df2, ignore_index = True)
                
        return df